# GitUp

In [ ]:
!git add .
!git commit -m "EIA to ds2"
!git push

In [1]:
# Libraries
from datashop import *
from eia_classes import *
import requests
import sqlite3
import pprint as pp
import pdb
dep = Depot()

In [96]:
key = 'm6HsQpjurALl27VpNGB1o4nAcr50o5wt'

example = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={}&api-key={}'.format('oil',key)

params ={
    'begin_date':'20000101',
    'end_date':'20200101',
    'q':'Saudi Arabia',
    'f1':['byline','headline'],
    'page':2,
    'news_desk':'Energy',
    'facet':True,
    'facet_fields':'document_type'


}
request = requests.get(example,params)
series_dict = json.loads(request.text)

In [97]:
headlines = []

In [ ]:
'abstract','document_type','headline'['main'],'headline'['print_headline'],'keywords'

In [ ]:
desc = ''
daily_price = EIA_Series('Daily Price','PET.RWTC.D')
spot_feat = Dataset(daily_price.series_frame)
dep.ingest(daily_price)

In [ ]:
desc = ''
daily_production = EIA_Series('Weekly Stocks','PET.WTTSTUS1.W')
dep.ingest(daily_production)

In [ ]:
dep.originals

In [ ]:
desc = 'US imports of crude oil, monthly'

monthly_imports = EIA_Series('Monthly Imports','PET.MCRIMUS1.M',desc,date_format='%Y%m' )
import_feat = Dataset(monthly_imports.series_frame)
import_feat['Monthly Imports'].show_report()
dep.ingest(monthly_imports)

In [ ]:
ds = Dataset(dep.originals)

In [ ]:
ds['Monthly Imports'].show_report()

## Dow Jones

In [ ]:
df_dow = pd.read_csv('data/dji.csv')

df_dow['Date']=pd.to_datetime(df_dow['Date'])
        
df_dow.set_index(df_dow['Date'],drop=True,inplace=True)

In [ ]:
df_dow['Close'].plot()

In [ ]:
df_dow['delta_dow']  = df_dow['Close'].diff()

X = df_dow[['delta_dow']]

y = daily_price.series_frame[['deltaDaily Price']]

merged = y.merge(X,how='inner',left_index=True,right_index=True)

In [ ]:
merged.head()

In [ ]:
x=merged['deltaDaily Price']
y=merged['delta_dow']
plt.scatter(x=x,y=y)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

model= LinearRegression()

In [ ]:
model.fit(x.reshape(-1,1),y)

# To-Do

1. Moving averages?
2. Correlation between price and:
3. Features to add/explore:
    a. Active wells?
    b. Refinery inputs?
    c. consumption?
    d. month?
    e. 
    


# SCRAPS

In [ ]:
\
df_eia=functools.reduce(lambda x,y: x.join(y,how='outer'),frame_list)
df_eia.fillna(axis=0,method='ffill',inplace=True)

conn = sqlite3.connect('oilstocks.db')
c = conn.cursor()

for col in df_eia:
    df_eia[col]=preprocessing.scale(df_eia[col])

df_eia.to_sql("scaledstocks",conn,if_exists='replace')
conn.commit()

df_eia.to_sql("crudestocks",conn,if_exists='replace')